## Setting
- Desilo key 생성

In [1]:
from desilofhe import Engine

engine = Engine(use_bootstrap=True)

secret_key = engine.create_secret_key()
public_key = engine.create_public_key(secret_key)
relinearization_key = engine.create_relinearization_key(secret_key)
conjugation_key = engine.create_conjugation_key(secret_key)
bootstrap_key = engine.create_bootstrap_key(secret_key)

Division(Inverse) Algorithm

- 나눗셈이나 역수를 진행하는 함수의 CKKS(Desilo) 근사 구현

In [2]:
#Division(Inverse) algorithm(depth: 2d)
def Inv(x, d, relineralization_key):  # 0 < x < 2
    a = engine.subtract(2, x)  # a = 2 - x
    b = engine.subtract(1, x)  # b = 1 - x
    for _ in range(d):
        b = engine.multiply(b, b, relineralization_key)  # b = b^2
        a = engine.multiply(a, engine.add(1, b), relineralization_key)  # a = a * (1 + b)

    return a


def main():
    data = [1, 2, 3]
    scale_down_factor = 3
    
    # 1. data의 첫번쨰 원소만 암호화한 후 Inv 실행
    val_scaled_down = [x / scale_down_factor for x in data]
    enc_data = engine.encrypt(val_scaled_down, public_key)

    # 2. Inv 연산 적용
    inv_val_scaled_down = Inv(enc_data, 5, relinearization_key)
    plt_inv_val = engine.decrypt(inv_val_scaled_down, secret_key)

    # 3. 각 요소별 결과 출력
    inv_results = [val for val in plt_inv_val[:len(data)]]
    for i, res in enumerate(inv_results):
        print(f"Inv({data[i]}/{scale_down_factor}) = {res}")



if __name__ == '__main__':
    main()


Inv(1/3) = 3.0000000000339138
Inv(2/3) = 1.5000000000044602
Inv(3/3) = 0.9999999999985149


Square root Algorithm

- Square root를 진행하는 함수의 CKKS(Desilo) 근사 구현

In [3]:
#Square root algorithm

import math

def sqrt(x, d, relinearization_key, conjugation_key, bootstrap_key):  # 0 <= x <= 1
    a = x
    b = engine.subtract(x, 1)  # b = x - 1
    
    for _ in range(d):
        # a = a * (1 - b / 2)
        factor = engine.subtract(1, engine.multiply(0.5, b))
        a = engine.multiply(a, factor, relinearization_key)

        # b = b^2 * (b - 3)/4
        b_squared = engine.square(b, relinearization_key)
        b_minus_3 = engine.subtract(b, 3)
        numerator = engine.multiply(b_squared, b_minus_3, relinearization_key)
        b = engine.multiply(numerator, 0.25)
        if factor.level < 3 or a.level < 3 or b.level < 3:
            a = engine.bootstrap(a,relinearization_key, conjugation_key, bootstrap_key)
            b = engine.bootstrap(b,relinearization_key, conjugation_key, bootstrap_key)
            print("Bootstrapping performed")
        return a


def main():
    data = [1, 2, 3]
    scale_down_factor = 4
    print(f"scaling factor = {scale_down_factor}")
    for i, val in enumerate(data):
        

        # 1. 원본 값 암호화 및 복호화 확인
        val_scaled_down = [x / scale_down_factor for x in data]
        print(f"\n=== data[{i}] = {val_scaled_down[i]} / 근사해야 하는 값 = {math.sqrt(val_scaled_down[i])} ===")
        enc_orig = engine.encrypt(val_scaled_down, public_key)

        # 2. sqrt 연산 적용
        inv_val_scaled_down = sqrt(enc_orig, 30, relinearization_key, conjugation_key, bootstrap_key)
        plt_inv_val = engine.decrypt(inv_val_scaled_down, secret_key)

        # 3. 각 요소별 결과 출력
        inv_results = [val for val in plt_inv_val[:len(data)]]
        print(f"sqrt({val_scaled_down[i]}) = {(inv_results[i])}") # 실수 부분만 추출 후 print


if __name__ == '__main__':
    main()

scaling factor = 4

=== data[0] = 0.25 / 근사해야 하는 값 = 0.5 ===
sqrt(0.25) = 0.3437500000449655

=== data[1] = 0.5 / 근사해야 하는 값 = 0.7071067811865476 ===
sqrt(0.5) = 0.6250000000065065

=== data[2] = 0.75 / 근사해야 하는 값 = 0.8660254037844386 ===
sqrt(0.75) = 0.843749999999468


Absolute value Algorithm
- 절댓값를 진행하는 함수의 CKKS(Desilo) 근사 구현

In [ ]:
#Absolute value algorithm

def sqrt(x, d, relinearization_key, conjugation_key, bootstrap_key):  # 0 <= x <= 1
    a = x
    b = engine.subtract(x, 1)  # b = x - 1
    
    for _ in range(d):
        # a = a * (1 - b / 2)
        factor = engine.subtract(1, engine.multiply(0.5, b))
        a = engine.multiply(a, factor, relinearization_key)

        # b = b^2 * (b - 3)/4
        b_squared = engine.square(b, relinearization_key)
        b_minus_3 = engine.subtract(b, 3)
        numerator = engine.multiply(b_squared, b_minus_3, relinearization_key)
        b = engine.multiply(numerator, 0.25)
        if factor.level < 3 or a.level < 3 or b.level < 3:
            a = engine.bootstrap(a,relinearization_key, conjugation_key, bootstrap_key)
            b = engine.bootstrap(b,relinearization_key, conjugation_key, bootstrap_key)
            print("Bootstrapping performed")
    return a

def main():
    data = [-2, 1, -3]
    scale_down_factor = 3
    for i, val in enumerate(data):
        val_scaled_down = [(x / scale_down_factor)**2 for x in data]
        # 1. 원본 값 암호화 및 복호화 확인
        enc_orig = engine.encrypt(val_scaled_down, public_key)

        # 2. Abs 연산 적용
        inv_val_scaled_down = sqrt(enc_orig, 30, relinearization_key, conjugation_key, bootstrap_key)
        plt_inv_val = engine.decrypt(inv_val_scaled_down, secret_key)
        inv_results = [val for val in plt_inv_val[:len(data)]]
        print(f"Abs({data[i]}) = {(inv_results[i].real)*scale_down_factor}") # 실수 부분만 추출 후 print
    return 0


if __name__ == '__main__':
    main()


=== data[0] = 0.4444444444444444 / 근사해야 하는 값 = 0.4444444444444444 ===
Bootstrapping performed
Bootstrapping performed
Bootstrapping performed
Bootstrapping performed
Bootstrapping performed
Bootstrapping performed
Bootstrapping performed
Bootstrapping performed
Abs(-2) = 1.996062787883651

=== data[1] = 0.1111111111111111 / 근사해야 하는 값 = 0.1111111111111111 ===
Bootstrapping performed
Bootstrapping performed
Bootstrapping performed
Bootstrapping performed
Bootstrapping performed
Bootstrapping performed
Bootstrapping performed
Bootstrapping performed
Abs(1) = 0.9980394411726492

=== data[2] = 1.0 / 근사해야 하는 값 = 1.0 ===
Bootstrapping performed
Bootstrapping performed
Bootstrapping performed
